# Generate Random numbers nPr with r between 2 values

In [1]:
#generate random permutations nPr = P(n,r) = n!/(n-r)!
import random
import itertools
import pandas as pd
import numpy as np

#constituency = 'Meath2002'
#constituency = 'DublinNorth2002'
#constituency = 'DublinWest2002'
#n = 14 #candidates (Meath 14, Dublin North 12, Dublin West 9)
#r = 7  #preference cast
#o = 25000 #number of permutations to be generated (25000, 20000, 10000)

(constituency, n, r, o, eucMax) = ('DublinWest2002', 9, 6, 40000, 44.5)
#(constituency, n, r, o, eucMax) = ('DublinWest2002', 9, 7, 20000, 47.2)
#(constituency, n, r, o, eucMax) = ('DublinWest2002', 9, 8, 20000, 50.0)
#(constituency, n, r, o, eucMax) = ('DublinNorth2002', 12, 7, 40000, 46.3)
#(constituency, n, r, o, eucMax) = ('DublinNorth2002', 12, 8, 40000, 49.0)
#(constituency, n, r, o, eucMax) = ('DublinNorth2002', 12, 9, 40000, 52.0)
#(constituency, n, r, o, eucMax) = ('Meath2002', 14, 7, 25000, 32.0)
#0(constituency, n, r, o, eucMax) = ('Meath2002', 14, 8, 25000, 37.3)
#(constituency, n, r, o, eucMax) = ('Meath2002', 14, 9, 25000, 40.2)

nlist = list(range(1, n+1)) 

#generate a list of a number o of random perumtations nPr 
randGenPer = random.sample(list(itertools.permutations(nlist, r)), o)

## Convert the permutation lists into a list of votes

In [2]:
#function to convert a permutation list into vote
def listToVote(listgen, num):
    voteList=[]
    for j in range(num):
        voteList.append(np.NaN)
    #print(voteList)

    for k in range(len(listgen)):
        voteList[listgen[k]-1] =k+1 
    return(voteList)
    

In [3]:
#convert each of the permutations generated into a vote
votesGen=[]
for row in randGenPer:
    #print(row)
    votesGen.append(listToVote(row,n))
#print(votesGen)

## Get columns form regular file

In [4]:
#read in regular file to get column name
#input file
#constituency="DublinNorth2002"
my_csv='../data/'+constituency+'.csv'
#read in data (setting 1st row as header)
dfR = pd.read_csv(my_csv, na_values=["Missing"], header=[0])

print(dfR.columns)

#drop the numbers column (#df=df.drop(['No.'], 1))
dfR = dfR.drop(dfR.columns[[0]], axis=1)  # df.columns is zero-based pd.Index
#reset index to start a 1 and not 0
dfR.index = dfR.index + 1
print(dfR.columns)

Index(['Unnamed: 0', 'GP', 'LB', 'FF', 'SP', 'FF.1', 'SF', 'PD', 'NP', 'FG'], dtype='object')
Index(['GP', 'LB', 'FF', 'SP', 'FF.1', 'SF', 'PD', 'NP', 'FG'], dtype='object')


## get political compass data 

In [5]:
#download compass data positioning irish political parties on the political spectrum
pcData='../data/pc.csv'

#set column names
col_names = ['party', 'xaxis', 'yaxis']
#read in political compass data
dfPC = pd.read_csv(pcData, na_values=["Missing"], names=col_names)
print("Irish Parties on Political Compass")
print(dfPC)
points=[]
points = dfPC.party
x=[] #x-axis coordinates
y=[] #y-axis coordinates
pt=[]

#populate two lists with the x and y - coordinates
for p in points:
    pt.append(dfPC.party)
    x.append(dfPC.xaxis)
    y.append(dfPC.yaxis)

Irish Parties on Political Compass
  party  xaxis  yaxis
0    SP   -7.0   -2.5
1    SF   -4.5    1.0
2    GP   -1.0   -2.0
3    NP    0.0    0.0
4    LB    1.5   -0.5
5    FF    2.5    2.4
6    FG    3.5    2.5
7    PD    4.0    3.0


In [6]:
# get the party initials (1 characters) from the column names i.e. remove .1 .2 L Retc
#print(df.columns)
partiesList= list(dfR.columns)
#del partiesList[-1] #max values not added yet
print(partiesList)
partiesList2=[]
for pl in partiesList:
    #print(pl[:2] )
    partiesList2.append(pl[:2])
print(partiesList2)

['GP', 'LB', 'FF', 'SP', 'FF.1', 'SF', 'PD', 'NP', 'FG']
['GP', 'LB', 'FF', 'SP', 'FF', 'SF', 'PD', 'NP', 'FG']


In [7]:
def getEuclidVoteDetails(voteRow, dfPC):
    #populate 3 lists with text, x co-ord and y co-ord
    partyPCList = dfPC.party.tolist()
   # print("partyList",partyPCList)
    xi=[]
    yi=[]
    pp=[]
    for d in voteRow:
        inx=partyPCList.index(d)
        pp.append(dfPC.iloc[inx].tolist()[0])
        xi.append(dfPC.iloc[inx].tolist()[1])
        yi.append(dfPC.iloc[inx].tolist()[2])

   # for ip in range(len(pp)):
   #     print(f"{ip+1} - {pp[ip]} ({xi[ip]}, {yi[ip]})")

    #Calculate the eculidean distance a vote travels on the political spectrum
    dist = 0
    total_dist=0
    for pl in range(len(pp)):
    # initializing points in # numpy arrays      
        if pl != (len(pp)-1):
            point1 = np.array((xi[pl],yi[pl]))
            point2 = np.array((xi[pl+1],yi[pl+1]))

    #calculating Euclidean distance # using linalg.norm()
            dist = np.linalg.norm(point1 - point2)
            #print(f"Euclidean distance between vote {pl+1} and {pl+2} is {dist}")
            total_dist = total_dist + dist
 #   print(total_dist, len(pp))
    if len(pp) > 1:
        avg_dist = total_dist/(len(pp)-1)    
    else:
        avg_dist = total_dist
    #print("Total Distance Travelled on Political Compass as a vote transfers ", total_dist)
    #print("Average Distance Travelled on Political Compass of each vote transfers ", avg_dist)

    return(total_dist, avg_dist)

In [8]:
def getVotesParty(voterow, partieslegend):
    #convert a vote row into parties initials
    voteP=[]
    for v in range(len(voterow)):
        for w in range(len(voterow)):
            if voterow[w] == v+1:
                voteP.append(partieslegend[w])
    return(voteP)

## Make data frame from list and write to file

In [9]:
#generate data frame from list
dfRandGenPer = pd.DataFrame(votesGen, columns=dfR.columns)
#drop all dublicate rows keeping last one
dfRandGenPer.drop_duplicates(keep='last', inplace=True)

In [10]:
#add a column of the maximum count of the rows(highest preference vote cast)
dfRandGenPer['maxValues'] = dfRandGenPer.max(axis = 1)

#drop all dublicate rows keeping last one
dfRandGenPer.drop_duplicates(keep='last', inplace=True, subset=dfRandGenPer.columns.difference(['maxValues']))

In [11]:
print(dfRandGenPer.columns)
print(partiesList2)

Index(['GP', 'LB', 'FF', 'SP', 'FF.1', 'SF', 'PD', 'NP', 'FG', 'maxValues'], dtype='object')
['GP', 'LB', 'FF', 'SP', 'FF', 'SF', 'PD', 'NP', 'FG']


In [12]:
#calculate the euclidean distance for all rows
EuclidDist = []
AvgEucDist =[]
for dft5 in range(len(dfRandGenPer)):
    #print(dft5)
    #remove max values form 
    dftR = list(dfRandGenPer.iloc[dft5])
    del dftR[-1]
    #print("Call get Votes for Party in row")
    votePList=getVotesParty(dftR, partiesList2)
    #print(votePList)
    (tD, aD) = getEuclidVoteDetails(votePList, dfPC)
    #print(tD, aD)
    EuclidDist.append(tD)
    AvgEucDist.append(aD)

#print(type(EuclidDist))
dfRandGenPer['Euclid Dist'] = np.array(EuclidDist)
dfRandGenPer['Avg. Euc. Dist'] = np.array(AvgEucDist)
display(dfRandGenPer.tail(10))


,GP,LB,FF,SP,FF.1,SF,PD,NP,FG,maxValues,Euclid Dist,Avg. Euc. Dist
39990,1.0,NaN,3.0,5.0,2.0,NaN,4.0,NaN,6.0,6.0,31.165904,6.233181
39991,2.0,NaN,3.0,1.0,6.0,4.0,NaN,NaN,5.0,6.0,27.926100,5.585220
39992,NaN,1.0,3.0,2.0,NaN,5.0,4.0,6.0,NaN,6.0,34.378818,6.875764
39993,5.0,NaN,3.0,NaN,4.0,NaN,2.0,6.0,1.0,6.0,10.181001,2.036200
39994,1.0,NaN,5.0,3.0,NaN,2.0,NaN,6.0,4.0,6.0,25.011170,5.002234
39995,NaN,3.0,4.0,6.0,NaN,2.0,1.0,5.0,NaN,6.0,28.882934,5.776587
39996,NaN,2.0,1.0,6.0,NaN,NaN,4.0,3.0,5.0,6.0,21.985521,4.397104
39997,5.0,2.0,6.0,1.0,NaN,NaN,3.0,NaN,4.0,6.0,25.726632,5.145326
39998,NaN,1.0,NaN,4.0,NaN,3.0,2.0,5.0,6.0,6.0,29.068647,5.813729
39999,2.0,NaN,1.0,NaN,6.0,5.0,NaN,4.0,3.0,6.0,28.035800,5.607160


In [13]:
#add column with (Irregular = 0)
dfRandGenPer['Seq']=0

In [14]:
dfI=pd.DataFrame()
dfI=dfRandGenPer.loc[(dfRandGenPer['Euclid Dist'] > eucMax) & (dfRandGenPer['maxValues']==7.0)]
display(dfI)

,GP,LB,FF,SP,FF.1,SF,PD,NP,FG,maxValues,Euclid Dist,Avg. Euc. Dist,Seq


In [15]:
# float_format='%.0f' to get rid of the decimal 1 not 1.0
#write "irregular votes" to csv file
dfI.to_csv('../data/processed/'+constituency+'-'
                    +str(n)+'P'+str(r)+'-r-'+str(o)+'irreg.csv')

#dfRandGenPer.to_csv('../data/processed/'+constituency+'-'+str(n)+'P'+str(r)+'-r-'+str(o)+'irreg.csv',                     float_format='%.0f')
